In [51]:
import numpy as np
import tensorflow as tf
import keras
from keras_crf import CRFModel

In [18]:
inputSentence = input("Enter sentence with symptom descriptions:")
inputSentence

inputSentence = inputSentence.lower()

In [42]:
# Perform stemming on the sentence

from TglStemmer import stemmer

tokenizedSentence = stemmer('2', inputSentence, '1')
print(stemmer_result)


TAGALOG WORDS STEMMER....
[1 FileName] [2 RawString] [3 ShowInfo]
Chosen raw string as source.
ako : ako
ako : ako =  {'word': 'ako', 'root': 'ako', 'prefix': [], 'infix': [], 'suffix': [], 'repeat': [], 'dupli': [], 'clean': []}
ay : ay
ay : ay =  {'word': 'ay', 'root': 'ay', 'prefix': [], 'infix': [], 'suffix': [], 'repeat': [], 'dupli': [], 'clean': []}
nagkakasipon : sipon
nagkakasipon : sipon =  {'word': 'nagkakasipon', 'root': 'sipon', 'prefix': ['nag'], 'infix': [], 'suffix': [], 'repeat': ['ka'], 'dupli': [], 'clean': ['ka']}
['ako', 'ay', 'sipon']


In [41]:
#from nltk.tokenize import word_tokenize
#tokenizedSentence = word_tokenize(inputSentence)
#tokenizedSentence

TypeError: expected string or bytes-like object

In [22]:
import json

folder_name = 'cfg'

with open("{}/{}.json".format(folder_name, "word_list")) as json_file:
    wordIdList = json.load(json_file)

with open("{}/{}.json".format(folder_name, "ner_config")) as json_file:
    ner_config = json.load(json_file)

with open("{}/{}.json".format(folder_name, "tags")) as json_file:
    tags = json.load(json_file)

with open("{}/{}.json".format(folder_name, "symptom_list")) as json_file:
    symptom_list = json.load(json_file)

with open("{}/{}.json".format(folder_name, "disease_list")) as json_file:
    disease_list = json.load(json_file)

with open("{}/{}.json".format(folder_name, "stopwords-tl")) as json_file:
    stopwords = json.load(json_file)


In [49]:
for stopword in stopwords:
    for word in tokenizedSentence:
        if (word == stopword):
            tokenizedSentence.remove(word)
tokenizedSentence

['sipon']

In [45]:
END_IDX = ner_config["n_words"] - 1
UNK_IDX = ner_config["n_words"]

sentence2idx = []

for word in tokenizedSentence:
    wordFound = False
    for key, val in wordIdList.items():
        if (word == key):
            wordFound = True
            sentence2idx.append(val)
    if (not wordFound):
        sentence2idx.append(UNK_IDX)
while (len((sentence2idx)) < ner_config["maxlen"]):
    sentence2idx.append(END_IDX)
sentence2idx


[12, 15, 15, 15, 15, 15]

In [52]:
#from keras_crf import CRFModel
#tf.keras.utils.register_keras_serializable(CRFModel)
ner_model = tf.keras.models.load_model('ner_model.h5')
ner_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 6, 300)            4800      
                                                                 
 bidirectional (Bidirectiona  (None, 6, 600)           1442400   
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 6, 600)            2882400   
                                                                 
 time_distributed (TimeDistr  (None, 6, 3)             1803      
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 6, 3)              0         
                                                                 
Total params: 4,331,403
Trainable params: 4,331,403
Non-

In [53]:
type(sentence2idx)

list

In [54]:
p = ner_model.predict(np.array([sentence2idx]))
print(p)
p = np.argmax(p, axis=-1)

input_symptom_list = []

print("{:14}: {}".format("Word", "Pred"))
for idx, (w, pred) in enumerate(zip(sentence2idx, p[0])):
    print("{:14}: {}".format(tokenizedSentence[idx], tags[pred]))
    if (tags[pred] == 'B-SYMPTOM'):
        temp_token = pred
        symptom_word = tokenizedSentence[idx]
        if ((temp_token + 1) == 'I-SYMPTOM'):
            temp_token += 1
            while ((temp_token) == 'I-SYMPTOM'):
                symptom_word + " " + tokenizedSentence[temp_token]
                temp_token += 1
        input_symptom_list.append(symptom_word)
    if (idx == len(tokenizedSentence) - 1):
        break

1/1 [==============================] - 1s 923ms/step
[[[1.4541462e-07 9.9809867e-01 1.9012702e-03]
  [4.8298449e-10 8.0073333e-01 1.9926667e-01]
  [1.9673314e-10 9.9857020e-01 1.4298313e-03]
  [1.8746738e-12 1.0000000e+00 9.3601771e-10]
  [1.3143480e-14 1.0000000e+00 8.6464055e-13]
  [4.2234600e-15 1.0000000e+00 5.2388915e-13]]]
Word          : Pred
sipon         : O


In [55]:
input_symptom_list

[]

In [20]:
# Match each symptom with the trained symptom list
input_to_boolean = []

for symptom in symptom_list:
    symptomInList = False
    for input_symptom in input_symptom_list:
        print (input_symptom, " ", symptom)
        if (input_symptom == symptom): 
            symptomInList = True
    if (symptomInList):
        input_to_boolean.append(1)
    else:
        input_to_boolean.append(0)

input_to_boolean


hilo   ubo
hilo   sipon
hilo   hilo
hilo   lagnat
hilo   sore throat


[0, 0, 1, 0, 0]

In [21]:
import pandas as pd

symptom_columns = []
for symptom in symptom_list:
    symptom_columns.append([symptom,],)

cols = pd.MultiIndex.from_arrays(symptom_columns)

input_frame = pd.DataFrame(symptom_columns, columns=[cols])


In [22]:
import pickle

with open('naiveBayes.pkl', 'rb') as f:
    naiveBayes = pickle.load(f)


In [42]:
from sklearn.preprocessing import RobustScaler

input_array = np.array(input_to_boolean)
input_array = input_array.reshape(1, -1)

scaler = RobustScaler()
test = scaler.fit_transform(input_array)

prediction_result = naiveBayes.predict_proba(input_array)
round(prediction_result[0][3], 6)


0.999977